# Grist code extraction
It extracts the list of codes that have been processed at a certain date, one-off script

In [ ]:
import pandas as pd
from dotenv import load_dotenv
import os

load_dotenv()

grist_existing_codes_pathfile = os.environ['GRIST_EXISTING_CODES_PATHFILE']
db_existing_codes_pathfile = os.environ['EXISTING_CODES_PATHFILE_2025']
consolidated_codes_output_pathfile = os.environ['CONSOLIDATED_CODES_OUTPUT_PATHFILE']

df_grist_codes = pd.read_csv(grist_existing_codes_pathfile, dtype=str, delimiter=',')
df_existing_codes_from_db = pd.read_csv(db_existing_codes_pathfile, dtype=str, delimiter=',')

In [ ]:
df_grist_codes = df_grist_codes[['Code pass Sport', 'Fait le 1.12']]
df_grist_codes.rename(columns={
    'Fait le 1.12': 'status',
    'Code pass Sport': 'code'
}, inplace=True)

df_grist_codes['status'] = df_grist_codes['status'].str.strip().str.lower()
df_grist_codes['code'] = df_grist_codes['code'].replace('"', '', regex=False).replace('\n', '', regex=False).replace('\r', '', regex=False)

In [ ]:
status = ('t','02/12','03/12')
df_grist_codes = df_grist_codes[df_grist_codes['status'].isin(status)]['code']
df_grist_codes = df_grist_codes.replace(r'[\n\r]', '', regex=True)
df_grist_codes.dropna(inplace=True)

In [ ]:
df_consolidated_codes = pd.concat([df_grist_codes, df_existing_codes_from_db]).reset_index(drop=True)
df_consolidated_codes.drop_duplicates(subset=['code'], inplace=True, keep='first')
df_consolidated_codes.to_csv(consolidated_codes_output_pathfile, index=False, sep=';')